## Feature WOE && IV


In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from tabulate import tabulate
import gc
import warnings

from time import sleep

warnings.filterwarnings("ignore")

In [2]:
train = pd.read_parquet("../data/processed/train_with_labels.parquet")

In [4]:
# cat_cols = ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']
# num_cols = [col for col in train.columns if col not in cat_cols and col not in ["customer_ID", "S_2"]]

In [6]:
# cont_features = [col for col in train.columns if col in num_cols and len(np.unique(train[col]))>10]

In [7]:
def iv_woe(data, feature, target, bins=10, show_woe=False, show_iv=False):
    # Empty Dataframe
    iv_df, woeDF = pd.DataFrame(), pd.DataFrame()
    # Extract Column Names
    cols = [feature]
    # Run WOE and IV on all the independent variables
    for ivars in cols:
        if (data[ivars].dtype.kind in "bifc") and (len(np.unique(data[ivars])) > 10):
            binned_x = pd.qcut(data[ivars], bins, duplicates="drop")
            data[ivars] = binned_x
            data[ivars] = data[ivars].astype(str)
            d0 = pd.DataFrame({"x": binned_x, "y": data[target]})
            d0 = d0.astype({"x": str})
        else:
            d0 = pd.DataFrame({"x": data[ivars], "y": data[target]})
            data[ivars] = data[ivars]

    d = d0.groupby("x", as_index=False, dropna=False).agg({"y": ["count", "sum"]})
    d.columns = ["Cutoff", "N", "Good"]
    d.insert(loc=0, column="Variable", value=ivars)

    d["% of Good"] = np.maximum(d["Good"], 0.5) / d["Good"].sum()
    d["Bad"] = d["N"] - d["Good"]
    d["% of Bad"] = np.maximum(d["Bad"], 0.5) / d["Bad"].sum()
    d["WoE"] = np.log(d["% of Good"] / d["% of Bad"])
    d["IV"] = d["WoE"] * (d["% of Good"] - d["% of Bad"])

    temp = pd.DataFrame(
        {"Variable": [ivars], "IV": [d["IV"].sum()]}, columns=["Variable", "IV"]
    )
    iv_df = pd.concat([iv_df, temp], axis=0)
    woeDF = pd.concat([woeDF, d], axis=0)
    return iv_df, woeDF

In [8]:
variaveis = train.columns
# df1 = {}
l = 0
for col in variaveis:
    if col == "target" or col == "customer_ID" or col == "S_2":
        continue
    else:
        print("WoE e IV: {}".format(col))
        iv, df = iv_woe(train, col, "target")
        # df1[l] = df
        # df1[l]["nome"] = "WOE_"+col
        print(l)
        l = l + 1
        # print(tabulate(df, headers="keys"))
        # print("IV score: {:.2f}".format(iv))
        print("\n")
        i = 0
        for i in range(0, len(df)):
            train.loc[train[col] == df.iloc[i, 1], col] = df.iloc[i, 7]
        del df, iv
        gc.collect()

WoE e IV: P_2
0


WoE e IV: D_39
1


WoE e IV: B_1
2


WoE e IV: B_2
3


WoE e IV: R_1
4


WoE e IV: S_3
5


WoE e IV: D_41
6


WoE e IV: B_3
7


WoE e IV: D_42
8


WoE e IV: D_43
9


WoE e IV: D_44
10


WoE e IV: B_4
11


WoE e IV: D_45
12


WoE e IV: B_5
13


WoE e IV: R_2
14


WoE e IV: D_46
15


WoE e IV: D_47
16


WoE e IV: D_48
17


WoE e IV: D_49
18


WoE e IV: B_6
19


WoE e IV: B_7
20


WoE e IV: B_8
21


WoE e IV: D_50
22


WoE e IV: D_51
23


WoE e IV: B_9
24


WoE e IV: R_3
25


WoE e IV: D_52
26


WoE e IV: P_3
27


WoE e IV: B_10
28


WoE e IV: D_53
29


WoE e IV: S_5
30


WoE e IV: B_11
31


WoE e IV: S_6
32


WoE e IV: D_54
33


WoE e IV: R_4
34


WoE e IV: S_7
35


WoE e IV: B_12
36


WoE e IV: S_8
37


WoE e IV: D_55
38


WoE e IV: D_56
39


WoE e IV: B_13
40


WoE e IV: R_5
41


WoE e IV: D_58
42


WoE e IV: S_9
43


WoE e IV: B_14
44


WoE e IV: D_59
45


WoE e IV: D_60
46


WoE e IV: D_61
47


WoE e IV: B_15
48


WoE e IV: S_11
49


WoE e IV: D_62
50


WoE e IV: D_

In [9]:
train.isna().sum()

S_2       0
P_2       0
D_39      0
B_1       0
B_2       0
         ..
D_142     0
D_143     0
D_144     0
D_145     0
target    0
Length: 190, dtype: int64

In [10]:
train.isna().sum().sum()

0

In [12]:
train.to_parquet("../data/processed/train_full_woebalanced.parquet")